# transfertab

> Allow transfer learning using structured data.

In [ ]:
#hide
from transfertab.core import *
from nbdev.showdoc import *
from fastai.tabular.all import *

ModuleNotFoundError: No module named 'transfertab'

## Install

`pip install transfertab`

## How to use

We have two fastai learners `learn1` and `learn2`  created, with models having embedding layers for their categorical variables.

In [ ]:
#hide
path = untar_data(URLs.ADULT_SAMPLE)
df1 = pd.read_csv(path/'adult.csv')
splits1 = RandomSplitter(valid_pct=0.2)(range_of(df1))
to1 = TabularPandas(df1, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits1)
dls1 = to1.dataloaders(bs=64)
learn1 = tabular_learner(dls1, metrics=accuracy)

#We add robot to our "race" column
new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
                        [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
                        columns=df1.columns)
df2 = df1.copy()
df2 = df2.append(new_rows, ignore_index=True)
df2.tail()

splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)



NameError: name 'untar_data' is not defined

In [ ]:
learn1, learn2, learn1.model

(<fastai.tabular.learner.TabularLearner at 0x7fa113e953a0>,
 TabularModel(
   (embeds): ModuleList(
     (0): Embedding(10, 6)
     (1): Embedding(17, 8)
     (2): Embedding(8, 5)
     (3): Embedding(16, 8)
     (4): Embedding(7, 5)
     (5): Embedding(6, 4)
     (6): Embedding(3, 3)
   )
   (emb_drop): Dropout(p=0.0, inplace=False)
   (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (layers): Sequential(
     (0): LinBnDrop(
       (0): Linear(in_features=42, out_features=200, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): LinBnDrop(
       (0): Linear(in_features=200, out_features=100, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (2): LinBnDrop(
       (0): Linear(in_features=100, out_features=2, bias=True)
     )
   )
 ))

For transferring the embeddings from `learn1` to `learn2`, we first instantiate the `TabTransfer` class with the learners and the categorical variables to transfer.

In [ ]:
tabobj = TabTransfer(learn2)

NameError: name 'TabTransfer' is not defined

Now, we can just call the transfer function to start the process.

In [ ]:
tabobj.init_from_json("../data/adults.json")

NameError: name 'tabobj' is not defined

In [ ]:
tabobj.transfer(verbose = True)

mean is tensor([ 0.0027, -0.0050, -0.0009,  0.0026,  0.0011, -0.0004],
       grad_fn=<MeanBackward1>) for Parameter containing:
tensor([[-0.0156, -0.0057,  0.0052,  0.0033,  0.0026,  0.0061],
        [-0.0070, -0.0103, -0.0070,  0.0056,  0.0013,  0.0010],
        [ 0.0118, -0.0094, -0.0015, -0.0127, -0.0066, -0.0163],
        [ 0.0042, -0.0039, -0.0106, -0.0068, -0.0124, -0.0183],
        [-0.0010, -0.0093, -0.0049,  0.0077,  0.0119, -0.0080],
        [-0.0022, -0.0082,  0.0072,  0.0114,  0.0166,  0.0077],
        [-0.0007,  0.0102,  0.0074, -0.0060, -0.0025, -0.0020],
        [ 0.0077, -0.0047,  0.0007,  0.0156,  0.0053,  0.0077],
        [ 0.0181, -0.0021,  0.0005,  0.0061, -0.0098,  0.0119],
        [ 0.0113, -0.0067, -0.0055,  0.0019,  0.0041,  0.0066]],
       requires_grad=True)
0, <class 'int'>
Transferring weights for class #na#, cat workclass from previous weights
old weight for class is tensor([ 0.0068, -0.0076,  0.0119,  0.0084,  0.0189,  0.0128],
       grad_fn=<SliceBackw